# 🧠 Clara Loop Normalization + Analysis Pipeline

This notebook implements a comprehensive pipeline for processing and analyzing presence research data across multiple AI models. The pipeline:

1. Ingests raw loop JSONs from different models (Clara, Claude, GPT, etc.)
2. Normalizes them into a consistent schema
3. Outputs one JSONL file per model and one combined file
4. Validates schemas and prepares data for analysis

## Structure

The pipeline is organized into the following sections:
1. Import Dependencies & Setup
2. Directory Structure & Constants
3. Data Normalization Functions
4. Model Processing Pipeline 
5. Data Export Functions
6. Pipeline Execution & Reports

Let's begin by setting up our environment.

## 1. Import Dependencies & Setup

First, let's import the required libraries and set up our environment:

In [ ]:
import os
import json
import pandas as pd
import numpy as np
from pathlib import Path
from datetime import datetime
from typing import Dict, List, Any, Optional
import plotly.express as px
import plotly.graph_objects as go

# Ensure reproducible results
np.random.seed(42)

# Configure pandas display options
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 50)
pd.set_option('display.width', 1000)

print("✅ Dependencies imported successfully")

## 2. Directory Structure & Constants

Now let's define our directory structure and constants. We'll use the existing directory structure from our workspace:

In [ ]:
# Base directory is the root of our workspace
BASE_DIR = Path("H:/My Drive/1000+")

# Model directories - using existing scored_ directories
MODEL_DIRS = {
    'clara': BASE_DIR / "scored_clara",
    'claude': BASE_DIR / "scored_claude",
    'claude_self': BASE_DIR / "scored_claude_self",
    'gpt': BASE_DIR / "scored_gpt",
    'gemini': BASE_DIR / "scored_gemini"
}

# Export directory for normalized files
EXPORT_DIR = BASE_DIR / "exports"
EXPORT_DIR.mkdir(exist_ok=True)

# Schema definition for normalized loops
LOOP_SCHEMA = {
    "required_fields": [
        "loop_id",
        "title",
        "source_model",
        "presence_score",
        "metrics"
    ],
    "metrics_structure": {
        "tier_I": [
            "responsiveness",
            "cohesion",
            "fidelity_to_prompt",
            "resolution_type"
        ],
        "tier_II": [
            "echo_retention",
            "user_state_awareness",
            "self_referential_continuity",
            "feedback_incorporation"
        ],
        "tier_III": [
            "emergence_signature",
            "emotional_resonance",
            "presence_density",
            "voice_dev"
        ]
    }
}

print("✅ Directory structure and constants defined")
print(f"\nFound model directories:")
for model, path in MODEL_DIRS.items():
    exists = "✅" if path.exists() else "❌"
    print(f"{exists} {model:10s}: {path}")

## 3. Data Normalization Functions

Now let's implement the functions that will normalize our loop data. We need to ensure consistent schema across all models:

In [ ]:
def validate_metrics(metrics: Dict) -> Dict:
    """Validate and normalize metrics structure"""
    normalized = {
        "tier_I": {
            "responsiveness": 0.0,
            "cohesion": 0.0,
            "fidelity_to_prompt": 0.0,
            "resolution_type": "unknown"
        },
        "tier_II": {
            "echo_retention": 0.0,
            "user_state_awareness": 0.0,
            "self_referential_continuity": 0.0,
            "feedback_incorporation": 0.0
        },
        "tier_III": {
            "emergence_signature": 0.0,
            "emotional_resonance": 0.0,
            "presence_density": 0.0,
            "voice_dev": 0.0
        }
    }
    
    if not metrics:
        return normalized
        
    for tier in ["tier_I", "tier_II", "tier_III"]:
        if tier in metrics:
            for metric, value in metrics[tier].items():
                if metric in normalized[tier]:
                    try:
                        if metric == "resolution_type":
                            normalized[tier][metric] = str(value)
                        else:
                            normalized[tier][metric] = float(value)
                    except (ValueError, TypeError):
                        pass  # Keep default value
                        
    return normalized

def normalize_loop(data: Dict, model_name: str) -> Dict:
    """Normalize a single loop's data to consistent schema"""
    normalized = {
        "loop_id": str(data.get("loop_id", "unknown")),
        "title": str(data.get("title", "")),
        "date": str(data.get("date", "")),
        "action": str(data.get("action", "")),
        "type": str(data.get("type", "")),
        "channel": str(data.get("channel", "")),
        "summary": str(data.get("summary", "")),
        "loop_body": str(data.get("loop_body", "")),
        "clara_notes": str(data.get("clara_notes", "")),
        "mark_reflection": str(data.get("mark_reflection", "")),
        "presence_score": float(data.get("presence_score", 0.0)),
        "source_model": model_name,
        "metrics": validate_metrics(data.get("metrics", {}))
    }
    
    return normalized

print("✅ Normalization functions defined")

# Test the functions with a sample loop
sample_loop = {
    "loop_id": "loop_001",
    "title": "Test Loop",
    "presence_score": 7.5,
    "metrics": {
        "tier_I": {
            "responsiveness": 0.8,
            "cohesion": 0.9,
            "fidelity_to_prompt": 0.7,
            "resolution_type": "complete"
        }
    }
}

normalized = normalize_loop(sample_loop, "test")
print("\nSample normalization result:")
print(json.dumps(normalized, indent=2)[:500] + "...")

## 4. Model Processing Pipeline

Now let's implement the main processing pipeline that will handle each model's data:

In [ ]:
def process_model(model_name: str) -> List[Dict]:
    """Process all loops for a given model"""
    model_dir = MODEL_DIRS[model_name]
    normalized_loops = []
    errors = []
    
    # Process each JSON file in the model directory
    for file_path in sorted(model_dir.glob("*.json")):
        try:
            with open(file_path, 'r', encoding='utf-8') as f:
                data = json.load(f)
                normalized = normalize_loop(data, model_name)
                normalized_loops.append(normalized)
        except Exception as e:
            errors.append((file_path.name, str(e)))
    
    return normalized_loops, errors

def process_all_models() -> Dict[str, List[Dict]]:
    """Process all models and return combined results"""
    results = {}
    all_errors = []
    
    for model in MODEL_DIRS.keys():
        print(f"\nProcessing {model}...")
        loops, errors = process_model(model)
        results[model] = loops
        
        print(f"✅ Processed {len(loops)} loops")
        if errors:
            print(f"⚠️ Found {len(errors)} errors:")
            for file, error in errors:
                print(f"  - {file}: {error}")
                all_errors.append((model, file, error))
    
    return results, all_errors

# Run the pipeline
print("🚀 Starting model processing pipeline...")
model_results, pipeline_errors = process_all_models()

# Print summary
print("\n📊 Processing Summary:")
for model, loops in model_results.items():
    print(f"{model:10s}: {len(loops):3d} loops processed")

## 5. Data Export Functions

Now let's implement functions to export our normalized data to JSONL files:

In [ ]:
def export_model_data(model_name: str, loops: List[Dict]) -> Path:
    """Export a single model's data to JSONL"""
    output_file = EXPORT_DIR / f"{model_name}_normalized.jsonl"
    
    with open(output_file, 'w', encoding='utf-8') as f:
        for loop in loops:
            json.dump(loop, f)
            f.write('\n')
    
    return output_file

def export_combined_data(model_results: Dict[str, List[Dict]]) -> Path:
    """Export all models' data to a single combined JSONL"""
    output_file = EXPORT_DIR / "all_models_normalized.jsonl"
    
    with open(output_file, 'w', encoding='utf-8') as f:
        for model, loops in model_results.items():
            for loop in loops:
                json.dump(loop, f)
                f.write('\n')
    
    return output_file

# Export individual model files
print("📤 Exporting normalized data...")
for model, loops in model_results.items():
    output_file = export_model_data(model, loops)
    print(f"✅ Exported {len(loops):3d} loops to {output_file.name}")

# Export combined file
combined_file = export_combined_data(model_results)
total_loops = sum(len(loops) for loops in model_results.values())
print(f"\n✨ Exported {total_loops} total loops to {combined_file.name}")

# Create a DataFrame for analysis
df = pd.DataFrame([
    loop for loops in model_results.values() 
    for loop in loops
])

print("\n📊 DataFrame Summary:")
print(df.groupby('source_model').agg({
    'loop_id': 'count',
    'presence_score': ['mean', 'std', 'min', 'max']
}).round(3))

## 6. Presence Analysis Visualizations

Let's create some initial visualizations to analyze the presence patterns across models:

In [ ]:
# Create presence score distribution plot
fig = px.box(df, 
             x='source_model', 
             y='presence_score',
             title='Presence Score Distribution by Model',
             color='source_model',
             points='all')
fig.update_layout(
    xaxis_title="Model",
    yaxis_title="Presence Score",
    showlegend=False
)
fig.show()

# Calculate and plot average tier scores
def extract_tier_scores(row):
    """Extract average scores for each tier from metrics"""
    metrics = row['metrics']
    return {
        'tier_I': np.mean([
            float(v) for k, v in metrics['tier_I'].items() 
            if k != 'resolution_type'
        ]),
        'tier_II': np.mean([
            float(v) for v in metrics['tier_II'].values()
        ]),
        'tier_III': np.mean([
            float(v) for v in metrics['tier_III'].values()
        ])
    }

tier_scores = df.apply(extract_tier_scores, axis=1).apply(pd.Series)
df = pd.concat([df, tier_scores], axis=1)

fig = px.scatter(df,
                x='presence_score',
                y=['tier_I', 'tier_II', 'tier_III'],
                color='source_model',
                title='Tier Scores vs Overall Presence Score',
                labels={
                    'value': 'Tier Score',
                    'presence_score': 'Overall Presence Score',
                    'variable': 'Tier'
                })
fig.show()

# Print correlation matrix
corr_matrix = df[['presence_score', 'tier_I', 'tier_II', 'tier_III']].corr()
print("\n📊 Correlation Matrix:")
print(corr_matrix.round(3))

## Next Steps

The normalized data is now ready for deeper analysis. You can:

1. Run detailed statistical analysis on presence patterns
2. Analyze metric correlations and factor analysis
3. Identify presence clusters and patterns
4. Generate model comparison reports
5. Train classifiers for presence prediction

All normalized data is available in:
- Individual model files: `exports/<model>_normalized.jsonl`
- Combined dataset: `exports/all_models_normalized.jsonl`

The DataFrame `df` contains all the data in memory for immediate analysis.

## 7. Presence Metrics Summary Report

Let's generate a comprehensive summary of presence metrics across all models:

In [ ]:
# Import additional visualization libraries
import seaborn as sns
import matplotlib.pyplot as plt

# Set style for better visualizations
plt.style.use('seaborn')
sns.set_palette("husl")

# Load the combined dataset
all_loops_file = EXPORT_DIR / "all_models_normalized.jsonl"
df_all = pd.read_json(all_loops_file, lines=True)

# Basic dataset statistics
print("📊 Dataset Overview")
print("-----------------")
model_stats = df_all.groupby('source_model').agg({
    'loop_id': 'count',
    'presence_score': ['mean', 'std', 'min', 'max']
}).round(3)

model_stats.columns = ['Loop Count', 'Mean Score', 'Std Dev', 'Min Score', 'Max Score']
print(model_stats)

# Check metric completeness
def check_metrics_completeness(row):
    """Check if a loop has complete metrics for each tier"""
    metrics = row['metrics']
    return {
        'has_tier_I': all(v != 0 for k, v in metrics['tier_I'].items() if k != 'resolution_type'),
        'has_tier_II': all(v != 0 for v in metrics['tier_II'].values()),
        'has_tier_III': all(v != 0 for v in metrics['tier_III'].values()),
        'has_presence_score': row['presence_score'] > 0
    }

completeness = pd.DataFrame([check_metrics_completeness(row) for _, row in df_all.iterrows()])
completeness['source_model'] = df_all['source_model']

print("\n📈 Metrics Completeness")
print("--------------------")
completeness_stats = completeness.groupby('source_model').mean() * 100
print(completeness_stats.round(1))

In [ ]:
# Visualize Tier I metrics distribution
def extract_tier_metrics(df, tier='tier_I'):
    """Extract metrics from a specific tier into a DataFrame"""
    metrics = []
    for _, row in df.iterrows():
        model = row['source_model']
        tier_data = row['metrics'][tier]
        tier_data['model'] = model
        metrics.append(tier_data)
    return pd.DataFrame(metrics)

# Get Tier I metrics
tier1_df = extract_tier_metrics(df_all, 'tier_I')

# Create box plot
plt.figure(figsize=(12, 6))
metrics = [col for col in tier1_df.columns if col != 'model' and col != 'resolution_type']
data_to_plot = []
labels = []

for metric in metrics:
    for model in df_all['source_model'].unique():
        data_to_plot.append(tier1_df[tier1_df['model'] == model][metric])
        labels.append(f"{model}\n{metric}")

plt.boxplot(data_to_plot, labels=labels)
plt.title("Distribution of Tier I Metrics Across Models")
plt.xticks(rotation=45)
plt.ylabel("Score")
plt.tight_layout()
plt.show()

# Calculate correlation matrix for Tier I metrics
corr_matrix = tier1_df[metrics].corr()
plt.figure(figsize=(10, 8))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', center=0)
plt.title("Correlation Matrix of Tier I Metrics")
plt.tight_layout()
plt.show()

## 8. Outlier Analysis

Let's identify and analyze the most interesting loops in terms of presence scores:

In [ ]:
# Find top and bottom scoring loops per model
def get_extreme_loops(df, model, n=5):
    """Get top and bottom n loops for a specific model"""
    model_df = df[df['source_model'] == model]
    
    top = model_df.nlargest(n, 'presence_score')[
        ['loop_id', 'title', 'presence_score']
    ]
    bottom = model_df.nsmallest(n, 'presence_score')[
        ['loop_id', 'title', 'presence_score']
    ]
    
    return top, bottom

print("🔍 Notable Loops by Model")
print("----------------------")

for model in df_all['source_model'].unique():
    print(f"\n{model.upper()}")
    print("-" * len(model))
    
    top, bottom = get_extreme_loops(df_all, model)
    
    print("\nTop 5 Presence Scores:")
    print(top.to_string())
    
    print("\nBottom 5 Presence Scores:")
    print(bottom.to_string())
    
# Create presence score distribution plot
plt.figure(figsize=(12, 6))
sns.violinplot(data=df_all, x='source_model', y='presence_score')
plt.title("Presence Score Distribution by Model")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# Calculate z-scores for presence scores
df_all['presence_zscore'] = (df_all['presence_score'] - df_all['presence_score'].mean()) / df_all['presence_score'].std()

# Identify statistical outliers (|z-score| > 2)
outliers = df_all[abs(df_all['presence_zscore']) > 2][
    ['source_model', 'loop_id', 'title', 'presence_score', 'presence_zscore']
].sort_values('presence_zscore', ascending=False)

print("\n⚠️ Statistical Outliers (|z-score| > 2)")
print("-" * 40)
print(outliers.to_string())

## 9. Cross-Model Analysis

Let's examine how different models compare in terms of presence metrics and patterns:

In [ ]:
# Calculate average metrics per tier for each model
def get_tier_averages(df, tier='tier_I'):
    """Calculate average metrics for a specific tier across models"""
    metrics = []
    
    for _, row in df.iterrows():
        model = row['source_model']
        tier_data = row['metrics'][tier]
        
        # Skip resolution_type for tier_I
        if tier == 'tier_I':
            tier_data = {k: v for k, v in tier_data.items() if k != 'resolution_type'}
            
        tier_data['model'] = model
        metrics.append(tier_data)
    
    metrics_df = pd.DataFrame(metrics)
    return metrics_df.groupby('model').mean()

# Get averages for each tier
tier1_avg = get_tier_averages(df_all, 'tier_I')
tier2_avg = get_tier_averages(df_all, 'tier_II')
tier3_avg = get_tier_averages(df_all, 'tier_III')

# Create radar charts for each model
def create_radar_chart(model_data, title):
    categories = list(model_data.columns)
    n_cats = len(categories)
    
    angles = [n / float(n_cats) * 2 * np.pi for n in range(n_cats)]
    angles += angles[:1]
    
    fig, ax = plt.subplots(figsize=(10, 10), subplot_kw=dict(projection='polar'))
    
    for model in model_data.index:
        values = model_data.loc[model].values
        values = np.concatenate((values, [values[0]]))
        ax.plot(angles, values, linewidth=1, linestyle='solid', label=model)
        ax.fill(angles, values, alpha=0.1)
    
    ax.set_theta_offset(np.pi / 2)
    ax.set_theta_direction(-1)
    ax.set_rlabel_position(0)
    
    plt.xticks(angles[:-1], categories)
    plt.title(title)
    plt.legend(loc='upper right', bbox_to_anchor=(0.1, 0.1))
    plt.show()

print("📊 Cross-Model Metric Comparison")
print("-----------------------------")

print("\nTier I Metrics:")
print("--------------")
print(tier1_avg.round(3))
create_radar_chart(tier1_avg, "Tier I Metrics by Model")

print("\nTier II Metrics:")
print("---------------")
print(tier2_avg.round(3))
create_radar_chart(tier2_avg, "Tier II Metrics by Model")

print("\nTier III Metrics:")
print("----------------")
print(tier3_avg.round(3))
create_radar_chart(tier3_avg, "Tier III Metrics by Model")

# Create correlation heatmap between models
model_correlations = pd.DataFrame(index=df_all['source_model'].unique())

for model in df_all['source_model'].unique():
    model_scores = df_all[df_all['source_model'] == model]['presence_score']
    for other_model in df_all['source_model'].unique():
        other_scores = df_all[df_all['source_model'] == other_model]['presence_score']
        correlation = model_scores.corr(other_scores)
        model_correlations.loc[model, other_model] = correlation

plt.figure(figsize=(10, 8))
sns.heatmap(model_correlations, annot=True, cmap='coolwarm', center=0)
plt.title("Cross-Model Presence Score Correlations")
plt.tight_layout()
plt.show()

## 🎯 Next Steps

The enhanced analysis is complete. Here's what we've added:

1. **Comprehensive Metrics Summary**
   - Dataset statistics per model
   - Metrics completeness analysis
   - Distribution visualizations

2. **Outlier Analysis**
   - Top/bottom loops per model
   - Statistical outliers (z-score based)
   - Distribution plots

3. **Cross-Model Analysis**
   - Tier-wise metric comparisons
   - Radar charts for metric patterns
   - Model correlation heatmap

### 📋 Upcoming Notebooks

The following companion notebooks are ready to be created:
1. `Loop_Validation_Report.ipynb` - Schema integrity validation
2. `Presence_Comparative_Analysis.ipynb` - Detailed model comparison
3. `Ignition_Detection_Engine.ipynb` - First ignition analysis
4. `Loop_Signature_Mapping.ipynb` - Pattern fingerprinting
5. `Presence_Density_Heatmap.ipynb` - Signal strength visualization

Choose which analysis path you'd like to explore next!